#16章 ロジスティック回帰

　広く用いられている教師あり学習技術のひとつである。

　"回帰"という名前だが、分類問題を扱う。

##レシピ16.1 2クラス分類器の訓練
###問題
　簡潔なクラス分類モデルを訓練したい。
###解決策
　`scikit-learn`の`LogisticRegression`を用いて、ロジスティック回帰器を訓練する。

In [0]:
#ライブラリをロード
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

#データをロードし、特徴量を2つに制限
iris = datasets.load_iris()
features = iris.data[:100,:]
target = iris.target[:100]

#特徴量を標準化
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

#ロジスティック回帰器を作成
logistic_regression = LogisticRegression(random_state=0)

#ロジスティック回帰器を訓練
model = logistic_regression.fit(features_standardized, target)

In [0]:
iris

###議論
　ロジスティック回帰は2クラス分類器(つまりターゲットベクトルが2つの値のうち1つを取る)で広く用いられている。ロジスティック回帰では線形モデル($\beta_0$ + $\beta_1x$など)がロジスティック関数(シグモイド関数とも呼ばれる)$\frac{1}{1+e^{-(\beta_0+\beta_1x)}}$の中に組み込まれている。

$$
P(y_i = 1|X) = \frac{1}{1+e^{-(\beta_0+\beta_1x)}}
$$

 ここで、

$P(y_i=1|X)$: $i$番目の観測値$y_i$がクラスに属する確率

$X$: 訓練データ

$\beta_0, \beta_1$: 訓練時に学習されるパラメータ

ロジスティック関数の効果は、関数の出力値を0から1の間に制約し、確率として解釈できるようにすることである。
$P(y_i-1|X)$が0.5よりも大きければ、クラス1であると予測される。そうでなければクラス0と予測される。

`scikit-learn`では`LogisticRegression`としてロジスティック回帰が実装されている。モデルを訓練すれば、それを用いて観測値のクラスを予測できる。

In [0]:
#新たな観測値を作成
new_observation = [[.5, .5, .5, .5]]

#クラスを予測
model.predict(new_observation)

array([1])

この例では、観測値は1と予測されている。

さらに、観測値がそれぞれのクラスに属する確率を見ることもできる。

In [0]:
#予測確率を表示
model.predict_proba(new_observation)

array([[0.17738424, 0.82261576]])

この観測値は、クラス0に属する確率が18.8%、**クラス1**に属する確率が81.1%となっている。

##レシピ16.2 多クラス分類器の訓練
###問題
　　3つ以上のクラスに対するクラス分類器を訓練したい。
###解決策
　　`scikit-learn`の`LogisticRegression`を使って、1対その他法(one-vs-rest)か、多項法(multinomial)を用いてロジスティック回帰器を訓練する。

In [0]:
#ライブラリをロード
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

#データをロード
iris = datasets.load_iris()
features = iris.data
target = iris.target

#特徴量を標準化
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

#1対その他法ロジスティック回帰器を作成
logistic_regression = LogisticRegression(random_state=0, multi_class="ovr")

#ロジスティック回帰器を訓練
model = logistic_regression.fit(features_standardized, target)

In [0]:
new_observation = [[4.6, 2.7, 1.3, 0.1]]
model.predict(new_observation)

array([2])

In [0]:
model.predict_proba(new_observation)

array([[0.00467529, 0.17485605, 0.82046866]])

###議論
　ロジスティック回帰器そのものは2クラス分類器なので、3つ以上のクラスを持つターゲットベクトルを扱うことはできない。しかし、ロジスティック回帰には、多クラスを扱うための2種類の巧妙な拡張がなされている。

　1つ目は、1対その他法ロジスティック回帰(OVR: one-vs-rest logistic regression)と呼ばれるもので、個々のクラスに対してそのクラスに属するかどうかだけを判断する(これは2クラス分類になる)独立したモデルを訓練する。この方法は、個々のクラス分類問題(クラス0かどうか、など)は互いに独立であることを仮定している。

　もう一つの方法は、多項ロジスティック回帰(MLR: Multinomial logistic regression)と呼ばれるもので、「レシピ15.1 観測値の近傍の発見」で示した式のロジスティック関数を下記の式で置き換えたものである。
$$
P(y_i=k|X)=\frac{e^{\beta_kx_i}}{\sum_{j=1}^{K}e^{\beta_ix_i} \quad}
$$

ここで、

$P(y_i=k|X)$: $i$番目の観測値

$y_i$: クラス$k$に属する確率

$K$: クラスの総数

多項ロジスティック回帰の実用上のメリットは、`predict_proba`で得られる予測確率の信頼性が高い(つまり、よく較正されている)ことである。

※較正されている(calibrated)：予測確率の値が正答率と一致していることを指す。つまり、予測確率が70%の観測値に対する正答率が70%であるならば、較正されている。

##レシピ16.3 正則化によるバリアンスの削減
###問題
　ロジスティック回帰モデルのバリアンスを削減したい。
###解決策
　正則化強度を示すハイパーパラメータCを調節する。

In [0]:
#ライブラリをロード
from sklearn.linear_model import LogisticRegressionCV
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

#データをロード
iris = datasets.load_iris()
features = iris.data
target = iris.target

#特徴量を標準化
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

#ロジスティック回帰器を作成
logistic_regression = LogisticRegressionCV(
    penalty='l2', Cs=10, random_state=0, n_jobs=-1)

#ロジスティック回帰器を訓練
model = logistic_regression.fit(features_standardized, target)

In [0]:
model.predict(new_observation)

array([1])

In [0]:
model.predict_proba(new_observation)

array([[0.00467529, 0.17485605, 0.82046866]])

#議論
　正則化とは、複雑なモデルにペナルティを与えることで、バリアンスを低減する手法である。具体的には、最小化の対象となるロス関数にペナルティ項を追加する。ペナルティ項としては、L1やL2がよく用いられる。L1ペナルティは下のようになる。
$$
\alpha\sum_{j=1}^{p}|\hat{\beta_j}| \quad
$$
ここで

$\hat{\beta_j}$: $p$個ある特徴量のうち$j$番目の特徴量に対応する学習されるパラメータ(重み)

$\alpha$: 正則化強度を示すハイパーパラメータ

L2は以下のようになる。
$$
\alpha\sum_{j=1}^{p}\hat{\beta^2_j} \quad
$$
$\alpha$を大きくすると、パラメータの値が大きい(つまりモデルが複雑である)ことに対するペナルティが増大する。scikit-learnでは、一般的な手法に従って、$\alpha$ではなく$C$を指定する。ここでは$C$は正則化強度$\alpha$の逆数、すなわち$C=\frac{1}{\alpha}$となる。ロジスティック回帰を使いながらバリアンスを削減するには、$C$をハイパーパラメータとして扱い、最良のモデルを与える$C$の値を探索すればよい。scikit-learnでは、`LogisticRegressionCV`クラスを用いて効率的に$C$を最適化することができる。`LogisticRegressionCV`のパラメータ`Cs`には$C$の探索範囲を浮動小数点のリストとして与えることもできるし、候補値の個数を整数で指定することもできる。後者の場合には、-10,000から10,000の範囲から対数スケールで候補値が抽出される。

　残念ながら`LogisticRegressionCV`は、ペナルティ項にまたがった探索はできない。つまり、最良のペナルティ項とパラメータを同時に探索することはできない。それを行うには、「12章 モデル選択」で述べた非効率的なモデル選択手法をを使う必要がある。